## Packages

In [17]:
import pandas as pd
from kafka import KafkaProducer
from json import dumps
from time import sleep
import json
from config import ip
import requests
from datetime import datetime, timedelta


## Kafka Producer

In [2]:
producer = KafkaProducer(bootstrap_servers=[f'{ip}:9092'],
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [3]:
while True:


    dt = datetime.now()
    dt.strftime("%Y-%m-%d %H:%M:%S")

    url = "https://api.exchange.coinbase.com/products/TRAC-USD/ticker"
    response = requests.get(url)
    TRAC = json.loads(response.text)
    TRAC['time'] = dt.strftime("%Y-%m-%d %H:%M:%S")
    producer.send('price', value=TRAC)
    sleep(15)



KeyboardInterrupt: 

In [4]:
producer.flush()

## Scratch

In [14]:
# url = "https://api.coinbase.com/v2/exchange-rates?currency=BTC"
# response = requests.get(url)
# ex_rates = json.loads(response.text)
# ex_rates['data']['rates']['USD']

# producer.send('Price', value={'Lee': 'Simpson'})


# url = "https://api.pro.coinbase.com/products/BTC-USD/candles/1m"
# response = requests.get(url)
# ex_rates = json.loads(response.text)
# ex_rates

[[1688602500, 30446.5, 30452.99, 30451.37, 30450.79, 0.44673506],
 [1688602440, 30449.12, 30456.71, 30449.12, 30451.15, 2.13673048],
 [1688602380, 30448.62, 30464.92, 30464.67, 30448.62, 1.77536886],
 [1688602320, 30460.28, 30466.54, 30460.28, 30464.68, 1.67434168],
 [1688602260, 30460.03, 30470.02, 30468.9, 30461.65, 4.27054502],
 [1688602200, 30468.32, 30477.85, 30475.24, 30468.39, 0.85581976],
 [1688602140, 30464.86, 30483.44, 30466.78, 30477.35, 4.41579771],
 [1688602080, 30464.24, 30469.99, 30466.62, 30466.84, 2.05907679],
 [1688602020, 30466.09, 30478.43, 30477.99, 30466.54, 1.3294388],
 [1688601960, 30473.3, 30483.79, 30483.79, 30477.97, 3.62189427],
 [1688601900, 30480.29, 30490.78, 30480.29, 30483.79, 10.54305187],
 [1688601840, 30479.64, 30482.83, 30481.55, 30480.29, 0.58364316],
 [1688601780, 30479.58, 30482.99, 30482.08, 30482.99, 1.34638522],
 [1688601720, 30481.32, 30487.48, 30483.84, 30482.6, 4.39307989],
 [1688601660, 30483.15, 30492.65, 30492.49, 30484.68, 1.34143836],

In [15]:
import http.client
import json

conn = http.client.HTTPSConnection("api.exchange.coinbase.com")
payload = ''
headers = {
  'Content-Type': 'application/json'
}
conn.request("GET", "/products/:product_id/candles", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"message":"User-Agent header is required."}


In [35]:
apiUrl = "https://api.pro.coinbase.com"
sym = "TRAC-USD"
barsize = "300"
timeEnd = datetime.now()
delta = timedelta(minutes=10)
timeStart = timeEnd - delta

timeStart = timeStart.isoformat()
timeEnd = timeEnd.isoformat()

parameter = {
    'start': timeStart,
    'end': timeEnd,
    'granularity': barsize
}

data = requests.get(apiUrl + "/products/" + sym + "/candles",             params=parameter,
        headers={"content-type": "application/json"}).json()

df = pd.DataFrame(data,
                  columns=['time', 'low', 'high', 'open', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='s')
df

,time,low,high,open,close,volume
0,2023-07-05 20:35:00,0.2516,0.2526,0.2517,0.2526,1100.3
1,2023-07-05 20:15:00,0.2512,0.2535,0.2528,0.2512,3607.5
2,2023-07-05 20:05:00,0.2521,0.2525,0.2525,0.2521,1355.0
3,2023-07-05 19:25:00,0.2514,0.2536,0.2514,0.2536,52.3
4,2023-07-05 19:15:00,0.2514,0.2520,0.2520,0.2514,6031.0
...,...,...,...,...,...,...
78,2023-07-04 21:25:00,0.2466,0.2485,0.2474,0.2485,736.0
79,2023-07-04 21:10:00,0.2456,0.2461,0.2461,0.2456,614.1
80,2023-07-04 21:00:00,0.2458,0.2462,0.2462,0.2458,168.2
81,2023-07-04 20:50:00,0.2461,0.2495,0.2463,0.2466,13695.9
